In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
input_file = 'income_data.txt'
df = pd.read_csv(input_file, header=None, na_values=' ?', skipinitialspace=True)

In [4]:
df.dropna(inplace=True)

In [5]:
# Виділення міток класу
df.iloc[:, -1] = df.iloc[:, -1].map({'<=50K': 0, '>50K': 1})

In [6]:
# Розділення ознак (X) і міток (y)
X = df.iloc[:, :-1]
y = df.iloc[:, -1].astype(int)

In [9]:
# Обробка категоріальних змінних
label_encoders = {}
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Збереження енкодерів

In [10]:
# Масштабування числових даних
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
models = {
    'Linear SVM': SVC(kernel='linear', random_state=0),
    'RBF SVM': SVC(kernel='rbf', random_state=0),
    'Poly SVM (degree=3)': SVC(kernel='poly', degree=3, random_state=0),
    'Sigmoid SVM': SVC(kernel='sigmoid', random_state=0)
}

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'Модель: {model.kernel}')
    print(f'Акуратність: {accuracy*100:.2f}%')
    print(f'Точність: {precision*100:.2f}%')
    print(f'Повнота: {recall*100:.2f}%')
    print(f'F1-міра: {f1*100:.2f}%')
    print('-' * 40)

In [16]:
for name, model in models.items():
    train_and_evaluate(model, X_train, y_train, X_test, y_test)

Модель: linear
Акуратність: 80.90%
Точність: 80.39%
Повнота: 80.90%
F1-міра: 77.21%
----------------------------------------
Модель: rbf
Акуратність: 84.62%
Точність: 83.81%
Повнота: 84.62%
F1-міра: 83.61%
----------------------------------------
Модель: poly
Акуратність: 84.34%
Точність: 83.56%
Повнота: 84.34%
F1-міра: 83.05%
----------------------------------------
Модель: sigmoid
Акуратність: 76.05%
Точність: 75.71%
Повнота: 76.05%
F1-міра: 75.87%
----------------------------------------


In [17]:
input_data = ['37', 'Private', '215646', 'HS-grad', '9', 'Never-married', 
              'Handlers-cleaners', 'Not-in-family', 'White', 'Male', 
              '0', '0', '40', 'United-States']

In [18]:
# Перетворення нового зразка на формат, який приймає модель
input_data_encoded = []
for i, col in enumerate(df.columns[:-1]):
    if col in label_encoders:
        input_data_encoded.append(label_encoders[col].transform([input_data[i]])[0])
    else:
        input_data_encoded.append(float(input_data[i]))

input_data_encoded = scaler.transform([input_data_encoded])

In [19]:
# Передбачення кожною моделлю
print('\nРезультати передбачення для нового зразка:')
for name, model in models.items():
    predicted_class = model.predict(input_data_encoded)[0]
    print(f'{name}: {">50K" if predicted_class == 1 else "<=50K"}')


Результати передбачення для нового зразка:
Linear SVM: <=50K
RBF SVM: <=50K
Poly SVM (degree=3): <=50K
Sigmoid SVM: <=50K
